# Python parallel computing

<br/>
<div align="center">18th of June, 2021</div>
<br/>
<div align="center">
    Thomas Arildsen<br/>
    <a href="mailto:tari@its.aau.dk">tari@its.aau.dk</a>
<div/>
<br/>
<div align="center">
Dept. of Electronic Systems<br/>
Aalborg University
</div>

*Partly based on material by Torben Larsen, T. Arildsen and T. L. Jensen*

# Python parallel computing

## Agenda

* Using the `multiprocessing` module
* Using the `concurrent.futures` module
* Somewhat different: `dask`

## Introduction

- We have looked a bit at theoretical details on parallel computing.
- We first focus on parallel computing across *one* physical computer (i.e. multiple CPUs / CPU cores with shared memory)
- Now, what are the possibilities in Python?

### Processes vs threads

**Process**

*In computing, a process is an instance of a computer program that is being executed. It contains the program code and its current activity. Depending on the operating system (OS), a process may be made up of multiple threads of execution that execute instructions concurrently.* - [WikiPedia](https://en.wikipedia.org/wiki/Process_%28computing%29)

**Thread**

*A thread of execution is the smallest sequence of programmed instructions that can be managed independently by a scheduler, which is typically a part of the operating system. The implementation of threads and processes differs between operating systems, but in most cases a thread is a component of a process. Multiple threads can exist within one process, executing concurrently and sharing resources such as memory, while different processes do not share these resources. In particular, the threads of a process share its executable code and the values of its variables at any given time.* - [WikiPedia](https://en.wikipedia.org/wiki/Thread_%28computing%29)

### Threads in Python

- Python can have threads (see the `threading` module)

- Execution in Python is limited to one thread at a time due to the *Global Interpreter Lock* (GIL).

- This means that threads cannot execute in parallel

- Still, threads in Python can be a nice tool to improve performance of I/O-heavy operations, e.g. [this example](https://www.toptal.com/python/beginners-guide-to-concurrency-and-parallelism-in-python)

- However, we focus on parallel computing here

## `multiprocessing`

- Part of Python's standard library, a "classic" workhorse in parallel computing in Python

A few things to be aware of with `multiprocessing` (i.e. due to processes - not threads):

- ‘Heavy’ parallelism $\rightarrow$ process parallel meaning that we essentially make a number of Python interpreters running in parallel, each with their own full copy of whatever needed.
- This is administratively ‘heavy’ $\rightarrow$ copies of environments, variables, arrays, imports etc.
- Heavy on memory as we copy the full workspace for each parallel entity.
- But fortunately we can create shared variables and arrays.

This implies the following, which is particularly true for multiprocessing and similar...

- Due to administration let the processes WORK (not for a second but for longer time; experience will teach you).

Here we focus on a simple facility in `multiprocessing` for convenient parallel execution of the SPMD or MPMD type - a "pool" of "workers".

- Pool a worker pool, $W$, with $M$ workers is formed as $W = (W_0, \ldots, W_{M−1})$. Normally as many as you have cores available (or physical threads). In principle we can have as many as we want but it is inefficient to have more workers than we have physical threads. The workers do the heavy lifting.

...

- We form a number of tasks $N$ of which we should normally have $N \gg M$: $T = (T_0,\ldots,T_{N−1})$. If we have perfect control of the tasks and a priori knowledge we may sometimes be able to reduce $N$.
- We submit the tasks to a scheduler $S$, which then feeds the tasks to the pool of workers. Often a round robin approach is used where the next task in line is served once we have a free worker available.
- Possible to control the scheduling via asynchronous techniques if we wish.

Passing data *to* a process:
- Large amounts of data can be read from a file (e.g. HDF5) or can be shared between processes.
- Intermediate amounts of data can be shared, submitted via the process call or read from a file - whatever makes most sense.
- Small amounts of data should normally always be passed from the master process to the worker process via 
the call.

Passing data *from* a process:
- Best not to let a process write to shared memory - we have locks but they are not fool-proof.
- Large amounts of data can be stored in e.g. HDF5 files.
- Intermediate amounts of data can be stored in files or returned by the process to the master process.
- Small amounts of data should always be returned to the master process - may be collected and stored to a file later.

`multiprocessing` pool of workers:

In [ ]:
import multiprocessing as mp
M = mp.cpu_count()


In [ ]:
pool = mp.Pool(processes=M)
pool

As simple as that...

We have now created a pool of workers that we can run processes in parallel under.

### SPMD-style processing

Use the `multiprocessing.Pool.map_async` function.

- Similar to the native Python `map` function - i.e. apply a given function to list of different data items.

In [ ]:
import os, time

def _f(d):
    # Defines the f(d) function (f(d) is a single task)
    time.sleep(float(d)/10.)
    pid = os.getpid()
    print(" _f argument: {:2d}, process id: {:7d} ".format(d, pid))
    return pid

def _callback(dummy):
    # Defines the callback function
    print("Input to callback: {0}".format(dummy))
    print("Callback process id: {0}".format(os.getpid()))

Definining and starting the actual parallel computing

In [ ]:
if __name__ == '__main__': # We have to use this to make it work in this interactive interpreter
    print("Parent process id: {:7d}".format(os.getpid()))
    pool = mp.Pool(processes=M)
    result = pool.map_async(_f, (30 ,15 ,2), callback=_callback)
    pool.close()
    pool.join()
    print(result.get())

### Exercise

Does it really execute in parallel?
- Write a small script similar to the above example
- Verify that the tasks actually do execute in parallel by:
    - Writing a task function (`_f`) that makes the process sleep for a while
    - Apply the function to a small tuple of times using: `map` and `multiprocessing.Pool.map_async` and measure how long the total execution time is in each case.

### Some remarks

- `map_async` is *asynchronous*. That is it detaches from the worker processes and allows the main script to continue (possibly doing other things) while the tasks execute.
- The purpose of the callback is to signal when the processing is done.
- Closing the pool (`pool.close()`) means no additional tasks can be added to it.
- The call `pool.join()` means that the main process will halt here and wait until the workers are done.
- Finally, `result.get()` fetches the results of the tasks. This will also have the effect of halting the main process to wait for the results if the tasks are not done yet.

### Process status
- `result.ready()`: If the result is ready `True` is returned and `False` if the result is not ready at the time when the command is issued.
- `result.get()`: Wait until the result is ready and then return the result. This is an often used functionality when chained computations are needed - like when the result `result.get()` is used as input to another computation.
- `result.wait()`: Requests a wait until the result is ready. This means that a sequence of commands like `result.wait()` followed by `result.ready()` always returns `True`.
- `result.successful()`: If the result is available at the time the command is issued it does nothing, and if the result is not ready is raises an `AssertionError`.

### Chunk Size

* `map_async` splits the workload iterable (list, tuple etc.) into a number of chunks.
* Work is handed one chunk at a time to the workers.
* A specific chunk size can be requested with the `chunksize` argument - the resulting chunk size will be *approximately* this size (appr. due to possible rounding).
* Assume we have an integer amount $NL$ of data items we want to compute on $M$ workers.
* A good rule of thumb over a large range of combinations $N$ vs. $L$ is `chunksize`$ = \left\lceil \frac{N}{M} \right\rceil$

If you do not need the asynchronous functionality, the results can be gathered directly (waiting for them) by using `multiprocessing.Pool.map()` instead:

In [ ]:
import os, time

def _f(d):
    # Defines the f(d) function (f(d) is a single task)
    time.sleep(float(d)/10.)
    pid = os.getpid()
    print(" _f argument: {:2d}, process id: {:7d} ".format(d, pid))
    return pid

if __name__ == '__main__': # We have to use this to make it work in this interactive interpreter
    print("Parent process id: {:7d}".format(os.getpid()))
    pool = mp.Pool(processes=M)
    result = pool.map(_f, (30 ,15 ,2))
    print(result)

### MPMD-style processing

Use the `multiprocessing.Pool.apply_async` function.

In [ ]:
import os, time

def _f1(d):
    # Defines the f1(d) function (f(d) is a single task)
    return d + 1

def _f2(d):
    # Defines the f1(d) function (f(d) is a single task)c
    return d + 2

def _f3(d):
    # Defines the f1(d) function (f(d) is a single task)c
    return d + 3

def _callback(dummy):
    # Defines the callback function
    print("Input to callback: {0}".format(dummy))
    print("Callback process id: {0}".format(os.getpid()))

In [ ]:
if __name__ == '__main__': # We have to use this to make it work in this interactive interpreter
    print("Parent process id: {:7d}".format(os.getpid()))
    pool = mp.Pool(processes=M)
    results = []
    results.append(pool.apply_async(_f1, (1,), callback=_callback))
    results.append(pool.apply_async(_f2, (1,), callback=_callback))
    results.append(pool.apply_async(_f3, (1,), callback=_callback))
    pool.close()
    pool.join()
    print([result.get() for result in results])

### Exercise

Again, does it really execute in parallel?
- Write a small script similar to the first exercise
- Verify that the tasks actually do execute in parallel by:
    - Writing task functions (`_f1` etc.) that make the process sleep for a while
    - Apply each of the functions to a tuple containing one time value each by: calling the functions directly one after another and by using `multiprocessing.Pool.apply_async` and measure how long the total execution time is in each case.

### Shared memory

- Allows us to save memory as sharing is allowed between tasks.
- Can (in principle) be used for both read and write...
- Recommendation: Although a locking mechanism exists (see [multiprocessing.Lock](https://docs.python.org/3.7/library/multiprocessing.html#multiprocessing.Lock)) it is not foolproof - it only ensures that multiple tasks cannot concurrently write to the same memory.
- ...but what happens if one worker writes to the same data before another and you didn’t expect that order of writing?
- When using shared memory for reading set `lock=False` - otherwise you cannot have concurrent reads which is the point of it all.
- If you use shared arrays for read-only then remember a unit test to ensure data integrity.

Alternatives:
- Transfer variables via function calls where different types of data can be packed.
- For large amounts of data it is often preferred with selected fetching from e.g. an HDF5 file that allows parallel access.
- Sharing via dedicated communication channels.

We can share data across processes with the `multiprocessing.sharedctypes` module:  
*(NB: more convenient mechanism `multiprocessing.shared_memory` under way in [Python 3.8](https://docs.python.org/3/library/multiprocessing.shared_memory.html))*

In [ ]:
import multiprocessing as mp
from multiprocessing import sharedctypes
import numpy as np

def task_func(args):
    position, block_size = args
    np_array = np.ctypeslib.as_array(array_shared_by_processes)
    return np_array[position:position+block_size]

def _init(the_array):
    global array_shared_by_processes
    array_shared_by_processes = the_array

size = 12
blocksize = 3
shared_array = sharedctypes.RawArray(sharedctypes.ctypes.c_double, size)
np.ctypeslib.as_array(shared_array)[:] = np.arange(size)

if __name__ == '__main__':
    pool = mp.Pool(processes=4, initializer=_init, initargs=(shared_array, ))
    result = pool.map(task_func, list(zip(range(0, size, blocksize), [blocksize]*int(size/blocksize))))
    print(result)

*Above example inspired by: http://thousandfold.net/cz/2014/05/01/sharing-numpy-arrays-between-processes-using-multiprocessing-and-ctypes/*

Remarks:
- We create a `sharedctypes.RawArray` for the shared data.
- This can be interfaced as a NumPy array using `np.ctypeslib.as_array`.
- We use an initialiser function to declare this array as a global variable so that each of the task function processes can see it.
- In the task function the array is interfaced as a NumPy array again.
- The global variable is a trick we use, because the `shared_array` variable cannot easily be passed by `pool.map` directly.
- We *can* also write to the shared array, but this is what we do not recommend here for safety reasons.

### Summary

- `multiprocessing` provides a fairly straightforward mechanism to explicitly set up parallel computations of the SPMD or MPMD type.
- Due to the internal workings of Python (GIL), true parallelism is a bit "heavy" (separate processes each with their own copy of interpreter, variables etc.).
- Threads also possible (we have not shown how here), but only useful for I/O-heavy concurrency with a lot of waiting (not true parallelism).

## concurrent.futures
This is a new module that comes with Python 3 and has some of the same facilities as `multiprocessing`.
- `concurrent.futures.ProcessPoolExecutor` has more or less corresponding functionality to `multiprocessing.Pool`.

Advantages:
- If your tasks can happen to crash beyond Python's control, `multiprocessing.Pool` can hang indefinitely. You will not notice until running your tasks takes suspiciously long. This should not happen using `concurrent.futures.ProcessPoolExecutor` (https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.ProcessPoolExecutor)
- `concurrent.futures` has the same interface to threads and processes, so switching between threads and processes in your code is simple (not relevant for simultanous parallel processing, but possibly for I/O-bound tasks).

Speculation:
- `concurrent.futures.ProcessPoolExecutor` might eventually replace `multiprocessing.Pool` down the line, so it could be practical to know about.

### SPMD-style processing

Use `concurrent.futures.ProcessPoolExecutor.map`
- Similar to `multiprocessing.Pool.map_async` - i.e. apply a given function to iterable of different data items.

In [ ]:
from concurrent import futures
import os, time

def _f(d):
    # Defines the f(d) function (f(d) is a single task)
    time.sleep(float(d)/10.)
    pid = os.getpid()
    print(" _f argument: {:2d}, process id: {:7d} ".format(d, pid))
    return pid

if __name__ == '__main__': # We have to use this to make it work in this interactive interpreter
    print("Parent process id: {:7d}".format(os.getpid()))
    M = os.cpu_count()
    pool = futures.ProcessPoolExecutor(max_workers=M)
    future = pool.map(_f, (30 ,15 ,2))
    print('Something funny')
    pool.shutdown()
    print(list(future))

### MPMD-style processing

Use `concurrent.futures.ProcessPoolExecutor.submit`

Similar to `multiprocessing.Pool.apply_async`.

In [ ]:
def _f1(d):
    # Defines the f1(d) function (f(d) is a single task)
    return d + 1

def _f2(d):
    # Defines the f1(d) function (f(d) is a single task)
    return d + 2

def _f3(d):
    # Defines the f1(d) function (f(d) is a single task)
    return d + 3

if __name__ == '__main__': # We have to use this to make it work in this interactive interpreter
    pool = futures.ProcessPoolExecutor(max_workers=M)
    futurelist = []
    futurelist.append(pool.submit(_f1, 1))
    futurelist.append(pool.submit(_f2, 1))
    futurelist.append(pool.submit(_f3, 1))
    pool.shutdown()
    print([future.result() for future in futurelist])

### Summary

- `concurrent.futures` basically provides the same parallel computing facilities as `multiprocessing` as far as worker pools are concerned.
- `concurrent.futures` uses `multiprocessing` "under the hood", but has its own implementation of pools that in particular behaves better if your tasks can crash beyond the control of `multiprocessing.Pool` (cause segmentation fault).
- Similar interfaces for threads and processes makes it very easy to switch between the two. Remember that threads are still not truly parallel - like in `multiprocessing`.

### Exercise

`concurrent.futures`
- Write a small script implementing MPMD-style processing.
- Define two functions to submit as tasks:
    - `f1` adds a number to its input and returns the result.
    - `f2` multiplies its input by a number and returns the result.
- Apply the `f1` function to some scalar value as the first task.
- Apply as second task the `f2` function to the future returned by the first task.
- What happens?

## Dask

### What is Dask?

A flexible parallel computing library by Continuum Analytics (the company behind Anaconda).
- Provides several facilities that mimic existing popular Python packages for data analytics and numerical computing:
    - Mimics `array`s from NumPy.
    - Mimics `DataFrame`s from Pandas.
    - ...as well as other features...
- Here we focus on `array`s and the ability to parallellise tasks using the convenient `delayed` feature.

### Dask arrays

Dask arrays mimic NumPy arrays, i.e. they behave like NumPy arrays but only provide a subset of the functionality of NumPy arrays.
- Particularly good for large amounts of data that do not fit in memory.
- Data can reside on disk transparently and is conveniently loaded as needed.

### Creating Dask arrays

We can create Dask arrays from NumPy arrays

In [ ]:
from dask import array as da
import numpy as np

x = np.random.rand(1000,1000)
x

In [ ]:
type(x)

In [ ]:
dx = da.from_array(x, chunks=x.shape)
dx

In [ ]:
type(dx)

- Note that `dx` is not a NumPy array - it is a Dask array; behaves similarly but is not the same

- We cannot directly see the contents of `dx`
- They have not been computed yet

In [ ]:
dx[:5,:5].compute()

### Computing with Dask arrays

We can now manipulate the Dask array in NumPy-like ways.

**Example - matrix-vector product**

Create a vector:

In [ ]:
dy = da.random.random((1000,1), chunks=1000)

Form the product:

In [ ]:
dprod = dx.dot(dy)
dprod

In [ ]:
dprod[:10,0].compute()

### Chunk size matters

Notice the `chunks` argument to the Dask array creation functions. Like we saw with `multiprocessing.map_async`, the chunk size has a significant impact on the performance of our computations.

From the [Dask documentation](https://docs.dask.org/en/latest/array-chunks.html)
- A chunk should be small enough to fit comfortably in memory. We’ll have many chunks in memory at once.
- Chunks must be large enough so that computations on them take $\ll$ 1ms overhead per task that Dask scheduling incurs. A task should take longer than 100ms.
- Chunks should align with the computation that you want to do. For example if you plan to frequently slice along a particular dimension then it is more efficient with chunks aligned so that you access fewer chunks. If you want to add two arrays then its convenient if those arrays have matching chunk patterns.

**Example with different chunk sizes**

In [ ]:
dX = da.random.random((1000,1000),chunks=(10,100))
dy = da.random.random((1000,1),chunks=(100,1))
%timeit dX.dot(dy).compute()

In [ ]:
dX = da.random.random((1000,1000),chunks=(1000,100))
%timeit dX.dot(dy).compute()

### Storing Dask arrays

Dask arrays really are ideally suited for computations on large amounts of data that are too large to fit in memory.

Therefore, it includes functionality to easily store Dask arrays to disk (if they are not already - above we created our dask arrays in memory): the function `to_hdf5`

In [ ]:
da.to_hdf5('demofile.hdf5', '/X', dX)

In [ ]:
ls -lh demofile.hdf5

HDF5 files are a popular file format for large data in scientific computing settings.
- Very convenient to work with in several Python packages in data analysis, numerical computing etc. - for example Pandas, PyTables.
- Good choice of file format for interoperability with other applications.
- Also very convenient to work with in Dask.

Dask can also use other file formats for storing data.
- In fact it can use any file format that provides a NumPy slicing-like interface.

### Loading Dask arrays from disk

This is what really justifies the use of Dask. The ability to handle large arrays of data on disk by automatically only loading into memory what is necessary.
- Dask creates arrays based on the contents of files
- The Dask array sits as an automatic and convenient NumPy array-like interface on top of the file
- When performing operations on the arrays, Dask automatically loads the necessary data into memory

We load the previously stored 'demofile.hdf5' into a new Dask array.

This is done the same way as we created an array based on a NumPy array before. Now we need to open the HDF5 file first:

In [ ]:
import h5py

f = h5py.File('demofile.hdf5','r+')
dataset = f['X/']
dY = da.from_array(dataset, chunks=dataset.shape)
dY

In [ ]:
da.all(dX == dY).compute()

In [ ]:
f.close()

In [ ]:
da.all(dX == dY).compute()

### Exercise

Dask arrays:
- Write a small script that creates a new dask array.
- Fill the Dask array with random values using a function from `dask.array.random`.
- Store the array to an HDF5 file.
- Repeat the above for different chunk sizes and time how long it takes to generate and store in each case. Does it make any difference?

### Building parallel computations using `delayed`

- So far we have looked at Dask arrays for NumPy-like operations.
- Dask also has functionality for setting up parallel computations a bit like `multiprocessing.Pool.apply_async`.
- Use the `dask.delayed.delayed` function:
    - Wraps other functions and objects to be evaluated in parallel
    - Can also be used as a decorator (not show here)
- The terminology is conceptually similar to `concurrent.futures` where computations return "future" objects, the results of which are eventually computed using `.result()`. In Dask, computations return "delayed" objects whose results are eventually computed using `.compute()`.
    

**Example - sum all elements in an array**

In [ ]:
from dask.delayed import delayed
import time

def func(arr):
    return arr.sum()

t1 = time.time()
# This part only sets up the computations but does not do anything yet
darray = delayed(h5py.File('demofile.hdf5','r+')['/X'])
sums = []
for row in range(1000):
    sums.append(delayed(func)(darray[row, :]))
result = delayed(sum)(sums)

t2 = time.time()
# Here the actual computations are done    
tmp = result.compute()

t3 = time.time()
tmp

In [ ]:
t2 - t1

In [ ]:
t3 - t2

### Summary

- Dask effectively hides the setting up of parallel computations for our convenience.
- Not a framework designed for explicitly controlling the individual tasks, processes etc.
- Particularly suitable for computations on large data that does not fit into memory.
- NB: the examples show here were all toy examples of a size where using Dask probably does not demonstrate its true strength and using NumPy etc. directly might easily be more efficient.

## Other possibilities

- Check out Joblib: https://joblib.readthedocs.io

    - Easy interface to setting up parallel computations (uses `multiprocessing` behind the scenes).
    - Caching of results to disk
- Numerous possibilities listed here: https://wiki.python.org/moin/ParallelProcessing (the wiki may be somewhat out of date).